In [1]:
# !pip install evaluate nltk rouge_score scikit-learn --user

# Fetching Datasets

In [ ]:
#imports
import pandas as pd
import os, ipdb
import random, evaluate

import random
import string

# Fine-Tune Llama2-7b on custom dataset
import os, ipdb
from dataclasses import dataclass, field
from typing import Optional
import numpy as np
import torch, random
from datasets import DatasetDict, Dataset, load_dataset
from peft import AutoPeftModelForCausalLM, LoraConfig
from tqdm import tqdm
import wandb
from transformers import AutoModelForCausalLM, AutoTokenizer\
, BitsAndBytesConfig, HfArgumentParser, TrainingArguments, TrainerCallback

from trl import SFTTrainer
from trl.trainer import ConstantLengthDataset


# from ../evaluation_metrics import Metrics
seed = 42
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)

os.environ["TOKENIZERS_PARALLELISM"] = "false" # or "true", depending on your needs

# pd.options.display.max_rows , pd.options.display.max_columns  = 100,100  

device = 'cuda' if torch.cuda.is_available() else "cpu"
device

2023-09-06 13:31:42.368910: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-06 13:31:55.837723: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Check the data 


In [ ]:
# df = pd.read_parquet('train_tdm_f1_v1.parquet')#.iloc[:2000,:]
# df = pd.read_parquet('train_squad.parquet')

mode = "tdm"
# mode = "tdms"
train_path = f'../data/train_{mode}_f1_v2_short.parquet' 
validation_path = f'../data/dev_{mode}_f1_v2_short.parquet'

df = pd.read_parquet(f'{train_path}')

df

In [23]:
df['answer'].isin(['unanswerable']).any()

False

In [4]:
def generate_random_string(length=10):
    """Generate a random string of given length."""
    return ''.join(random.choice(string.ascii_letters) for _ in range(length))

def random_string_or_nothing(probability=0.5, length=10):
    """Generate a random string with a given probability or return an empty string."""
    if random.random() < probability:
        return generate_random_string(length)
    else:
        return ''

# Test the function
print(random_string_or_nothing())


In [5]:
df["prediction"] = df["answer"].apply(lambda x : str(x)+random_string_or_nothing(probability=0.5))

In [6]:
df['template_question'] = df['template_question'].astype(str)
df['answer'] = df['answer'].astype(str)
df['prediction'] = df['prediction'].astype(str)

df.head()

,template_question,answer,prediction
0,Value Prediction Network This paper proposes a...,[{'leaderboard': {'Dataset': 'Atari 2600 Seaqu...,[{'leaderboard': {'Dataset': 'Atari 2600 Seaqu...
1,Read this and answer the question. If the ques...,[{'leaderboard': {'Dataset': 'Atari 2600 Seaqu...,[{'leaderboard': {'Dataset': 'Atari 2600 Seaqu...
2,Value Prediction Network This paper proposes a...,[{'leaderboard': {'Dataset': 'Atari 2600 Seaqu...,[{'leaderboard': {'Dataset': 'Atari 2600 Seaqu...
3,Value Prediction Network This paper proposes a...,[{'leaderboard': {'Dataset': 'Atari 2600 Seaqu...,[{'leaderboard': {'Dataset': 'Atari 2600 Seaqu...
4,Value Prediction Network This paper proposes a...,[{'leaderboard': {'Dataset': 'Atari 2600 Seaqu...,[{'leaderboard': {'Dataset': 'Atari 2600 Seaqu...


In [7]:
# df.describe()
df.dtypes

template_question    object
answer               object
prediction           object
dtype: object

In [8]:
str(df.at[0, 'prediction'])

"[{'leaderboard': {'Dataset': 'Atari 2600 Seaquest', 'Metric': 'Score', 'Task': 'Atari Games'}}\n {'leaderboard': {'Dataset': 'Atari 2600 Amidar', 'Metric': 'Score', 'Task': 'Atari Games'}}\n {'leaderboard': {'Dataset': 'Atari 2600 Krull', 'Metric': 'Score', 'Task': 'Atari Games'}}\n {'leaderboard': {'Dataset': 'Atari 2600 Alien', 'Metric': 'Score', 'Task': 'Atari Games'}}\n {'leaderboard': {'Dataset': 'Atari 2600 Enduro', 'Metric': 'Score', 'Task': 'Atari Games'}}\n {'leaderboard': {'Dataset': 'Atari 2600 Ms. Pacman', 'Metric': 'Score', 'Task': 'Atari Games'}}\n {'leaderboard': {'Dataset': 'Atari 2600 Crazy Climber', 'Metric': 'Score', 'Task': 'Atari Games'}}\n {'leaderboard': {'Dataset': 'Atari 2600 Q*Bert', 'Metric': 'Score', 'Task': 'Atari Games'}}\n {'leaderboard': {'Dataset': 'Atari 2600 Frostbite', 'Metric': 'Score', 'Task': 'Atari Games'}}]rpoiVgRVIf"

In [24]:
dataset = DatasetDict.load_from_disk(f"../data/LLLM_DOCTEAT_TDM_ALL_TEMPLATE/fold2")
    
    # if args.streaming:
    #     print("Loading the dataset in streaming mode")
    #     valid_data = dataset.take(args.size_valid_set)
    #     train_data = dataset.skip(args.size_valid_set)
    #     train_data = train_data.shuffle(buffer_size=args.shuffle_buffer, seed=None)
    # else:
    
# dataset = dataset.train_test_split(test_size=0.005, seed=None)
train_data = dataset["train"]
valid_data = dataset["validation"]

train_data[0]

{'prompt': 'Value Prediction Network This paper proposes a novel deep reinforcement learning (RL) architecture, called Value Prediction Network (VPN), which integrates model-free and model-based RL methods into a single neural network. In contrast to typical model-based RL methods, VPN learns a dynamics model whose abstract states are trained to make option-conditional predictions of future values (discounted sum of rewards) rather than of future observations. Our experimental results show that VPN has several advantages over both model-free and model-based baselines in a stochastic environment where careful planning is required but building an accurate observation-prediction model is difficult. Furthermore, VPN outperforms Deep Q-Network (DQN) on several Atari games even with short-lookahead planning, demonstrating its potential as anew way of learning a good state representation. VPN has four more hyperparameters: 1) the number of predictions steps (k) during training, 2) the plan de

In [27]:
labels = []
preds = []
for i, valid_ex in tqdm(enumerate(valid_data), total=len(valid_data)):

    labels.append(valid_ex['answer'])
    preds.append(valid_ex['answer'])

    # if i == 2000:
    #     break
    
    # ipdb.set_trace()

100%|██████████| 35280/35280 [00:02<00:00, 15989.62it/s]


In [9]:
# rouge = evaluate.load('rouge')
# predictions = ["hello thre", "general kenobi"]
# references = ["hello there", "general "]
# results = rouge.compute(predictions=predictions,
#                         references=references)
# print(results)

rouge = evaluate.load('rouge')

results = rouge.compute(predictions=df['prediction'].tolist()[:1000],
                        references=df['answer'].tolist()[:1000])
print(results)

{'rouge1': 0.9918042175299275, 'rouge2': 0.991238744713131, 'rougeL': 0.9918065598603332, 'rougeLsum': 0.991804152725802}


In [30]:
clf_metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])

clf_metrics.compute(predictions=[1 if "unanswerable" == x else 0 for x in preds], 
                    references=[1 if "unanswerable" == x else 0 for x in labels]
                    # references=[1 for df['answer'].tolist()]
)


{'accuracy': 1.0, 'f1': 1.0, 'precision': 1.0, 'recall': 1.0}

In [5]:
str(df.at[5, 'answer'])

"[{'leaderboard': {'Dataset': 'Atari 2600 Seaquest', 'Metric': 'Score', 'Task': 'Atari Games'}}\n {'leaderboard': {'Dataset': 'Atari 2600 Amidar', 'Metric': 'Score', 'Task': 'Atari Games'}}\n {'leaderboard': {'Dataset': 'Atari 2600 Krull', 'Metric': 'Score', 'Task': 'Atari Games'}}\n {'leaderboard': {'Dataset': 'Atari 2600 Alien', 'Metric': 'Score', 'Task': 'Atari Games'}}\n {'leaderboard': {'Dataset': 'Atari 2600 Enduro', 'Metric': 'Score', 'Task': 'Atari Games'}}\n {'leaderboard': {'Dataset': 'Atari 2600 Ms. Pacman', 'Metric': 'Score', 'Task': 'Atari Games'}}\n {'leaderboard': {'Dataset': 'Atari 2600 Crazy Climber', 'Metric': 'Score', 'Task': 'Atari Games'}}\n {'leaderboard': {'Dataset': 'Atari 2600 Q*Bert', 'Metric': 'Score', 'Task': 'Atari Games'}}\n {'leaderboard': {'Dataset': 'Atari 2600 Frostbite', 'Metric': 'Score', 'Task': 'Atari Games'}}]"

In [8]:
# Number of CPU cores
workers = os.cpu_count()
workers

128

In [29]:
i = 0
for x in tqdm(labels):
    if "unanswerable" == x:
        ipdb.set_trace()
    else:
        i+=1
        
        # print(x)

  0%|          | 0/35280 [00:00<?, ?it/s]

> /tmp/ipykernel_430391/577914827.py(2)<module>()
      1 i = 0
----> 2 for x in tqdm(labels):
      3     if "unanswerable" == x:



ipdb>  print(x)


unanswerable


ipdb>  print(i)


23220


ipdb>  q


 66%|██████▌   | 23220/35280 [00:25<00:13, 903.91it/s]
